# 연습문제 (3) - 분산분석

In [9]:
from pandas import read_excel
from scipy.stats import f_oneway
from matplotlib import pyplot as plt
from pandas import read_excel, melt
from scipy.stats import shapiro, normaltest, ks_2samp, bartlett, fligner, levene, f_oneway, chi2_contingency
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
import sys
import os
import seaborn as sb
from pandas import DataFrame, read_excel, merge, concat
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())))
from helper import allTest #independence_test, equal_variance_test, normality_test

In [13]:
# 데이터 불러오기
df = read_excel("D:\\이지선\\project\\semi_project3\\한국복지패널_17차.xlsx")

df.head()

,가처분소득,태어난 연도,집의 가격,교육비-공교육비,교육비-사교육비,자녀수,피복신발비,보건의료비,교양오락비,교통비,통신비,식료품비_합산,광열수도비_합산
0,5958.0,1978,21000,88.0,67.0,1,17.0,34.0,20.0,51.0,25,120,33
1,1969.0,1961,25000,0.0,0.0,1,2.0,12.0,0.3,3.0,10,90,23
2,3406.0,1984,100000,0.0,0.0,1,3.0,5.0,1.0,5.0,7,81,17
3,7586.0,1972,17000,3.0,10.0,2,45.0,20.0,27.0,26.0,19,118,18
4,4494.0,1976,2500,0.0,42.0,2,3.0,12.0,2.0,16.0,18,128,14


In [14]:
df.columns


Index(['가처분소득', '태어난 연도', '집의 가격', '교육비-공교육비', '교육비-사교육비', '자녀수', '피복신발비',
       '보건의료비', '교양오락비', '교통비', '통신비', '식료품비_합산', '광열수도비_합산'],
      dtype='object')

In [15]:
df.dtypes

가처분소득       float64
태어난 연도        int64
집의 가격         int64
교육비-공교육비    float64
교육비-사교육비    float64
자녀수           int64
피복신발비       float64
보건의료비       float64
교양오락비       float64
교통비         float64
통신비           int64
식료품비_합산       int64
광열수도비_합산      int64
dtype: object

In [21]:
_, p_value = shapiro(df['가처분소득'])
_, p_value

(0.8515163660049438, 6.458034523863771e-35)

In [19]:
alpha = 0.05
for column in df.columns:
    _, p_value = shapiro(df[column])
    
    if p_value > alpha:
        print(f"{column}: 데이터는 정규 분포를 따를 가능성이 있습니다.")
    else:
        #print(f"{column}: 데이터는 정규 분포를 따르지 않을 가능성이 있습니다.")

SyntaxError: incomplete input (455182795.py, line 8)

p-value가 0.05보다 크므로 귀무가설을 기각할 수 없다. 즉, 통계적으로 유의미한 차이는 발견하지 못했다. 

`p-value`가 0.05보다 작으므로 귀무가설을 기각, 토마토 모종 성장에 차이가 발생하는 통계적으로 유의미한 결과가 나옴.

## 문제 3

두 가지 교육방법을 비교하기 위하여 능력이 비슷한 고등학교 학생 22명을 추출하고, 11명씩 두 집단으로 나누어 한 학기 동안 각각의 교육방법으로 교육시킨 후 치른 학기말 시험 성적이 다음과 같다.

이 자료를 이용하여 두 가지 교육방법에 대한 국어와 영어의 효과가 서로 다르다고 할 수 있는지 검정하라. (분석 목적에 맞는 데이터 재배치 과정이 필요합니다.)

![res/q33.png](res/q33.png)

> https://data.hossam.kr/E02/edu.xlsx

### 이원분산분석

In [ ]:
교육 = read_excel("https://data.hossam.kr/E02/edu.xlsx")
교육

In [ ]:
교육.melt(id_vars='교육방법')

### 정답풀이

교육방법에 따라 국어와 영어의 상관을 알아내는게 문제

In [ ]:
교육_국어 = 교육.filter(['교육방법','국어'])
교육_국어['교과목'] = '국어'
교육_국어.columns=['교육방법','점수','교과목']
# 교육_국어.rename(columns={'국어':'점수},inplace=True)
교육_국어

In [ ]:
교육_영어 = 교육.filter(['교육방법','영어'])
교육_영어['교과목'] = '영어'
교육_영어.columns=['교육방법','점수','교과목']
교육_영어

In [ ]:
교육_가공=concat([교육_국어,교육_영어])
교육_가공

In [ ]:
교육_가공['교과목'] = 교육_가공['교과목'].map({'국어': 1, '영어': 2})
df3 = 교육_가공.copy()
df3

In [ ]:
formula = '점수 ~ C(교육방법) * C(교과목)'
lm = ols(formula, 교육_가공).fit()
anova = anova_lm(lm)
anova

교육방법에 따라 유의미한 차이는 없다. 